In [ ]:
%pip install geopandas matplotlib plotly nbformat notebook ipywidgets ipyleaflet

In [9]:
import geopandas as gpd
import pandas as pd
from ipyleaflet import Map, GeoJSON, WidgetControl
import ipywidgets as widgets

# Load shapefile
world = gpd.read_file("./data/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")
europe = world[world['CONTINENT'] == 'Europe']

# Load OWID CO₂ data
co2_df = pd.read_csv("https://github.com/owid/co2-data/raw/master/owid-co2-data.csv")

# Define a list of European countries
european_countries = [
    'Albania', 'Andorra', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium',
    'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark',
    'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'Greece', 'Hungary',
    'Iceland', 'Ireland', 'Italy', 'Kazakhstan', 'Kosovo', 'Latvia', 'Liechtenstein',
    'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro',
    'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania',
    'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
    'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom', 'Vatican'
]

# Filter for European countries and years of interest
europe_emissions_raw = co2_df[
    (co2_df['country'].isin(european_countries)) &
    (co2_df['year'].isin([2019, 2021]))
]

# Pivot to wide format: one row per country
emissions_df = europe_emissions_raw.pivot(index='country', columns='year', values='co2').reset_index()
emissions_df.columns = ['ADMIN', 'co2_2019', 'co2_2021']
emissions_df = emissions_df.fillna(0)

# Optional country name fixes (match shapefile naming)
country_name_map = {
    "Czechia": "Czech Republic",
    "Bosnia and Herzegovina": "Bosnia and Herz.",
    "North Macedonia": "Macedonia",
    "Slovakia": "Slovak Republic",
    "Moldova": "Moldova",
    "Russia": "Russian Federation"
}
emissions_df['ADMIN'] = emissions_df['ADMIN'].replace(country_name_map)

# Merge with shapefile data
europe_emissions = europe.merge(emissions_df, how='left', on='ADMIN').fillna(0)

# Define GeoJSON styling
def get_geojson(year):
    def style_callback(feature):
        admin = feature['properties']['ADMIN']
        value = emissions_df.loc[emissions_df['ADMIN'] == admin, f'co2_{year}'].values
        emission = value[0] if len(value) else 0
        intensity = min(emission / emissions_df[f'co2_{year}'].max(), 1.0)
        color = f"rgba(255, 0, 0, {intensity})"
        return {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6,
        }

    return GeoJSON(
        data=europe_emissions.__geo_interface__,
        style_callback=style_callback,
        name=f"CO₂ {year}"
    )

# Create map and interactive controls
center = [54.0, 15.0]
m = Map(center=center, zoom=4)

layer_ref = [get_geojson("2019")]
m.add_layer(layer_ref[0])

dropdown = widgets.Dropdown(
    options=[('2019 (Pre-Pandemic)', '2019'), ('2021 (Post-Pandemic)', '2021')],
    value='2019',
    description='Year:',
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        m.remove_layer(layer_ref[0])
        new_layer = get_geojson(change['new'])
        m.add_layer(new_layer)
        layer_ref[0] = new_layer

dropdown.observe(on_change)

control = WidgetControl(widget=dropdown, position='topright')
m.add_control(control)

m


/var/folders/7z/jv8k1cd909v27l8ndwlg06g80000gn/T/ipykernel_97654/459343731.py:48: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  europe_emissions = europe.merge(emissions_df, how='left', on='ADMIN').fillna(0)


Map(center=[54.0, 15.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…